# loading packages and functions
run the cells below to load useful packages and functions

In [1]:
!pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [2]:
!pip install noisereduce

  Using cached noisereduce-2.0.1-py3-none-any.whl (15 kB)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import wave
from scipy.io import wavfile
import contextlib
import glob
import os 
from scipy.io import wavfile
import noisereduce as nr
import json
from pydub import AudioSegment



def interpret_wav(raw_bytes, n_frames, n_channels, sample_width, interleaved = True):

    if sample_width == 1:
        dtype = np.uint8 # unsigned char
    elif sample_width == 2:
        dtype = np.int16 # signed 2-byte short
    else:
        raise ValueError("Only supports 8 and 16 bit audio formats.")

    channels = np.frombuffer(raw_bytes, dtype=dtype)

    if interleaved:
        # channels are interleaved, i.e. sample N of channel M follows sample N of channel M-1 in raw data
        channels.shape = (n_frames, n_channels)
        channels = channels.T
    else:
        # channels are not interleaved. All samples from channel M occur before all samples from channel M-1
        channels.shape = (n_channels, n_frames)

    return channels

def get_start_end_frames(nFrames, sampleRate, tStart=None, tEnd=None):

    if tStart and tStart*sampleRate<nFrames:
        start = tStart*sampleRate
    else:
        start = 0

    if tEnd and tEnd*sampleRate<nFrames and tEnd*sampleRate>start:
        end = tEnd*sampleRate
    else:
        end = nFrames

    return (start,end,end-start)

def extract_audio(fname, tStart=None, tEnd=None):
    with contextlib.closing(wave.open(fname,'rb')) as spf:
        sampleRate = spf.getframerate()
        ampWidth = spf.getsampwidth()
        nChannels = spf.getnchannels()
        nFrames = spf.getnframes()

        startFrame, endFrame, segFrames = get_start_end_frames(nFrames, sampleRate, tStart, tEnd)

        # Extract Raw Audio from multi-channel Wav File
        spf.setpos(startFrame)
        sig = spf.readframes(segFrames)
        spf.close()

        channels = interpret_wav(sig, segFrames, nChannels, ampWidth, True)

        return (channels, nChannels, sampleRate, ampWidth, nFrames)

def convert_to_mono(channels, nChannels, outputType):
    if nChannels == 2:
        samples = np.mean(np.array([channels[0], channels[1]]), axis=0)  # Convert to mono
    else:
        samples = channels[0]

    return samples.astype(outputType)

def plot_specgram(samples, sampleRate, tStart=None, tEnd=None):
    plt.figure(figsize=(20,10))
    plt.specgram(samples, Fs=sampleRate, NFFT=1024, noverlap=192, cmap='nipy_spectral', xextent=(tStart,tEnd))
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

def plot_audio_samples(title, samples, sampleRate, tStart=None, tEnd=None):
    if not tStart:
        tStart = 0

    if not tEnd or tStart>tEnd:
        tEnd = len(samples)/sampleRate

    f, axarr = plt.subplots(2, sharex=True, figsize=(20,10))
    axarr[0].set_title(title)
    axarr[0].plot(np.linspace(tStart, tEnd, len(samples)), samples)
    axarr[1].specgram(samples, Fs=sampleRate, NFFT=1024, noverlap=192, cmap='nipy_spectral', xextent=(tStart,tEnd))
    #get_specgram(axarr[1], samples, sampleRate, tStart, tEnd)

    axarr[0].set_ylabel('Amplitude')
    axarr[1].set_ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.savefig(title+'.jpg')
    plt.show()



/Users/yirongx/opt/miniconda3/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def fir_high_pass(samples, fs, fH, N, outputType):
    # Referece: https://fiiir.com

    fH = fH / fs

    # Compute sinc filter.
    h = np.sinc(2 * fH * (np.arange(N) - (N - 1) / 2.))
    # Apply window.
    h *= np.hamming(N)
    # Normalize to get unity gain.
    h /= np.sum(h)
    # Create a high-pass filter from the low-pass filter through spectral inversion.
    h = -h
    h[int((N - 1) / 2)] += 1
    # Applying the filter to a signal s can be as simple as writing
    s = np.convolve(samples, h).astype(outputType)
    return s


def fir_low_pass(samples, fs, fL, N, outputType):
    # Referece: https://fiiir.com

    fL = fL / fs

    # Compute sinc filter.
    h = np.sinc(2 * fL * (np.arange(N) - (N - 1) / 2.))
    # Apply window.
    h *= np.hamming(N)
    # Normalize to get unity gain.
    h /= np.sum(h)
    # Applying the filter to a signal s can be as simple as writing
    s = np.convolve(samples, h).astype(outputType)
    return s

def fir_band_reject(samples, fs, fL, fH, NL, NH, outputType):
    # Referece: https://fiiir.com

    fH = fH / fs
    fL = fL / fs

    # Compute a low-pass filter with cutoff frequency fL.
    hlpf = np.sinc(2 * fL * (np.arange(NL) - (NL - 1) / 2.))
    hlpf *= np.blackman(NL)
    hlpf /= np.sum(hlpf)
    # Compute a high-pass filter with cutoff frequency fH.
    hhpf = np.sinc(2 * fH * (np.arange(NH) - (NH - 1) / 2.))
    hhpf *= np.blackman(NH)
    hhpf /= np.sum(hhpf)
    hhpf = -hhpf
    hhpf[int((NH - 1) / 2)] += 1
    # Add both filters.
    if NH >= NL:
        h = hhpf
        h[int((NH - NL) / 2) : int((NH - NL) / 2 + NL)] += hlpf
    else:
        h = hlpf
        h[int((NL - NH) / 2) : int((NL - NH) / 2 + NH)] += hhpf
    # Applying the filter to a signal s can be as simple as writing
    s = np.convolve(samples, h).astype(outputType)

    return s

def fir_band_pass(samples, fs, fL, fH, NL, NH, outputType):
    # Referece: https://fiiir.com

    fH = fH / fs
    fL = fL / fs

    # Compute a low-pass filter with cutoff frequency fH.
    hlpf = np.sinc(2 * fH * (np.arange(NH) - (NH - 1) / 2.))
    hlpf *= np.blackman(NH)
    hlpf /= np.sum(hlpf)
    # Compute a high-pass filter with cutoff frequency fL.
    hhpf = np.sinc(2 * fL * (np.arange(NL) - (NL - 1) / 2.))
    hhpf *= np.blackman(NL)
    hhpf /= np.sum(hhpf)
    hhpf = -hhpf
    hhpf[int((NL - 1) / 2)] += 1
    # Convolve both filters.
    h = np.convolve(hlpf, hhpf)
    # Applying the filter to a signal s can be as simple as writing
    s = np.convolve(samples, h).astype(outputType)

    return s


# combine data function
this is not mandatory to run following steps. just to reduce file number for each day to have better view.

In [3]:
# combine data
def combine(input_files,output_file):
    output_music = AudioSegment.empty()
    for f in input_files:
        input_music = AudioSegment.from_wav(f)
        output_music += input_music
    if os.path.exists(os.path.split(output_file)[0]):
        output_music.export(output_file, format="wav")
    else:
        os.makedirs(os.path.split(output_file)[0])
        output_music.export(output_file, format="wav")

# run combine step to your data
combine each day's files into one file. 

**You should define channel and birdname here.**

In [10]:
birdfolder = 'bird7292'
channel = 'chan6'
home_path = '/Volumes/RENATA_X/Vallentinlab'
step = 4 ## for 5 days we use one day as sample data 
days = glob.glob(home_path+'/'+birdfolder+'/'+channel+'/*')
dayinuse = [os.path.split(days[i])[1] for i in range(0,len(days),step)]

In [11]:
dayinuse = np.loadtxt(home_path+'/'+birdfolder+'_sorted/dayinuse.txt',dtype='str')

In [7]:
day_folder = [home_path+'/'+birdfolder+'/'+channel+'/'+day for day in dayinuse]
for day in day_folder[3:]:
    day_index = os.path.split(day)[-1]
    combine(glob.glob(day+'/*.wav'),home_path+'/'+birdfolder+'_sorted/'+channel+'/'+day_index+'.wav')

# denoise and filter
filter the recording use 1kHz-12kHz, then denoise the data. 

In [28]:
width_kernel = 500

for day in dayinuse[-4:]:
    wav_file = home_path+'/'+birdfolder+'_sorted/'+channel+'/%s.wav'%day
    channels, nChannels, sampleRate, ampWidth, nFrames = extract_audio(wav_file)
    samples = convert_to_mono(channels, nChannels, np.int16)
    print(wav_file)
    samples_filtered = fir_band_pass(samples, sampleRate, 1000, 12000, width_kernel, width_kernel, np.int16)[width_kernel-1:-width_kernel+1]
    
    reduced_noise = nr.reduce_noise(y=samples_filtered, sr=sampleRate)
    # perform noise reduction
    folder,fname = os.path.split(wav_file)
    
    wavfile.write(os.path.join(home_path+'/'+birdfolder+'_sorted/'+channel+'/',os.path.splitext(fname)[0] + '_filter_denoised.wav'), sampleRate, reduced_noise)

/Volumes/Yirong_work/Vallentinlab/bird7292_sorted/chan6/dph077.wav
/Volumes/Yirong_work/Vallentinlab/bird7292_sorted/chan6/dph081.wav
/Volumes/Yirong_work/Vallentinlab/bird7292_sorted/chan6/dph085.wav
/Volumes/Yirong_work/Vallentinlab/bird7292_sorted/chan6/dph091.wav


# sample data

In [12]:

for ori_file in glob.glob(home_path+'/'+birdfolder+'_sorted/'+channel+'/*_filter_denoised.wav'):
    channels, nChannels, sampleRate, ampWidth, nFrames = extract_audio(ori_file,0,180)
    samples = convert_to_mono(channels, nChannels, np.int16)
    wavfile.write(os.path.splitext(ori_file)[0]+'_sampled0.wav', sampleRate, samples)
